# Хабрахабр: мультиграммная тематическая модель

## Данные

- [habrahabr-dataset](https://github.com/romovpa/habrahabr-dataset)
- [Архивы с данными](https://yadi.sk/d/b13MG_XGfxVBp)

Распаковываем в `data` полный архив и добавляем к нему посчитанные N-граммы от Сергея Царькова: `data/ngrams`.

In [1]:
!ls data

habrahabr_ngrams  habr_posts  posts.csv


## Корпус для BigARTM

Делаем корпус мешков слов для обучения тематической модели в BigARTM.

In [2]:
import pandas
posts = pandas.read_csv('data/posts.csv', encoding='utf8') # таблица с базовой информацией о постах

Отсеиваем посты из единственного хаба "Чёрная дыра", поско

In [3]:
good_post_ids = posts.ix[posts['hubs'] != u'Чёрная дыра', 'post_id'].values

In [9]:
def post_to_bow(post_id):
    bow = []

    def word_to_token(word):
        return word.strip('#').replace(' ', '_').replace(':', '_').replace('|', '_')
        
    with open('data/habrahabr_ngrams/post_%d.xml' % post_id) as f:
        for line in f:
            line = line.decode('utf8')
            _, word, cnt = line.rstrip().split('\t')
            cnt = int(cnt)
            bow.append((word_to_token(word), cnt))
            
    return ' '.join(
        token + ('' if cnt == 1 else ':%d' % cnt)
        for token, cnt in bow
    )

In [12]:
good_post_ids.shape[0]

102776

In [15]:
import os

In [17]:
with open('data/habrahabr_ngrams.vw', 'w') as fout:
    for n, post_id in enumerate(good_post_ids):
        if not os.path.exists('data/habrahabr_ngrams/post_%d.xml' % post_id):
            continue
        line = ('post_%d |text ' % post_id) + post_to_bow(post_id) + '\n'
        fout.write(line.encode('utf8'))
        if n % 5000 == 0:
            print n

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000


In [18]:
!du -hs data/habrahabr_ngrams.vw

2.3G	data/habrahabr_ngrams.vw


In [11]:
print post_to_bow(100000)

юбилей:2 отмечает стотысячный_пост пост_поздравляю_жителей_ресурса пост_поздравляю праздник_дело стотысячный со_посетителей стотысячный_пост_поздравляю увлекательных любимый_ресурс_хабрахабр_отмечает для_праздник криков любимый_ресурс_хабрахабр_отмечает_юбилей ресурс_хабрахабр_отмечает торт_стороны любимый_ресурс под_названием под_пост гайдов торт хабрахабр знаменательным поздравляю_жителей_ресурса торт_со_стороны_посетителей ресурс_хабрахабр_отмечает_юбилей хабрахабр_отмечает увлекательных_статей жителей_ресурса юбилей_под_названием желаю нескучных_гайдов отмечает_юбилей пост_поздравляю_жителей дело жителей хабрахабр_отмечает_юбилей со_стороны любимый_ресурс_хабрахабр поздравляю стотысячный_пост_поздравляю_жителей_ресурса статей со_стороны_посетителей любимый ресурс_хабрахабр поздравляю_жителей ресурс:2 со_хабру нескучных стотысячный_пост_поздравляю_жителей


In [6]:
!head -n 50 data/habrahabr_ngrams/post_100000.xml

юбилей	#юбилей	2
отмечать	отмечает	1
пост стотысячный	#стотысячный пост	1
житель поздравлять пост ресурс	пост поздравляю жителей ресурса	1
поздравлять пост	пост поздравляю	1
дело праздник	#праздник дело	1
стотысячный	#стотысячный	1
посетитель	со посетителей	1
поздравлять пост стотысячный	стотысячный пост поздравляю	1
увлекательный	увлекательных	1
любимый отмечать ресурс хабрахабр	любимый ресурс хабрахабр отмечает	1
праздник	для праздник	1
крик	криков	1
любимый отмечать ресурс хабрахабр юбилей	любимый ресурс хабрахабр отмечает юбилей	1
отмечать ресурс хабрахабр	ресурс хабрахабр отмечает	1
сторона торт	торт стороны	1
любимый ресурс	#любимый ресурс	1
название	под названием	1
пост	под пост	1
гайд	гайдов	1
торт	#торт	1
хабрахабр	#хабрахабр	1
знаменательный	знаменательным	1
житель поздравлять ресурс	поздравляю жителей ресурса	1
посетитель сторона торт	торт со стороны посетителей	1
отмечать ресурс хабрахабр юбилей	ресурс хабрахабр отмечает юбилей	1
отмечать хабрахабр	хабрахабр отмечает	1
стат

In [4]:
!ls data/habrahabr_ngrams/ | head

post_100000_comments.xml
post_100000.xml
post_100006_comments.xml
post_100006.xml
post_100007_comments.xml
post_100007.xml
post_100008_comments.xml
post_100008.xml
post_100009_comments.xml
post_100009.xml
ls: write error: Broken pipe


## 